In [1]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel
from tqdm import tqdm
import pandas as pd
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

# Load the CSV file again
data = pd.read_csv('preprocessed_content_beauty.csv')

# Extracting each row from the 'preprocessed_content' column and storing it in a list
content_list = data['preprocessed_content'].tolist()

# Displaying the first few items in the list to verify
content_list[:2]

/home/winten/.pyenv/versions/3.10.11/envs/langchain_kurly/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['뷰티 뷰티 컬 리페 스타 추천 템 세 일 달려요 에일리 2023 2 11 18 35 url 복사 이웃 추가 본문 기타 기능 어제 부터 시작 된 뷰티 컬 리페 스타 소개 해요 단 7일 간 진행 되는 역대 급 쇼핑 찬스 라 놓치면 너무 너무 아까울 저 는 컬리 에서 진행 되는 걸 눈 여겨 보고 있다가 후다닥 들어갔는데 매일 3천 명에게 지급 되는 선착순 쿠폰 2 가지 중 1만원 권은 놓쳐서 4천원 권을 득템 해서 알뜰하게 성공했지요 새벽 배송 으로 유명한 컬리 이지만 오전 에 주문 했더니 세상 에 자정 전 에 도착 해서 깜짝 놀랐지 뭐 에요 ㅎㅎㅎ 단 7일 동안만 푸 짐 한 혜택 들 누리 며 그동안 사고싶었지만 가격 대가 부담스러웠던 뷰티 제품 들 쇼핑 하기 좋아요 2월 9일 부터 2월 16일 까지니까 놓치지말고 구입 하자고요 그럼 뷰티 컬리 페스 타의 혜택 은 어떤게 있을까 궁금하시죠 결제 혜택 으로는 제휴 사 혜택 오전 11시 마다 발급 되는 1만원 쿠폰 4천원 쿠폰 타임 어택 에서는 매일 오후 5시 마다 한정 수량 으로 구입 할 수 있는 인기 상품 들 50 세 일 빅세일 코너 에서는 스페셜 딜 플렉스 딜 플러스 딜 득템 딜 카테고리 특 가 최대 74 특가 로 역대 급 쇼핑 찬스 에요 인기 브랜드관 에서는 각 브랜드 마다 7일 한정 으로 할인 부터 증정 까지 다양한 행사 들 기다리고 있어요 우선 은 매일 오전 11시 마다 선착순 으로 다운 받아서 사용 할 수 있는 쿠폰 놓치지말고 받으시길 추천 해요 첫날 에는 깜빡 잊고 11시 반쯤 입장 했더니 1만원 쿠폰은 품절 더라고요 ㅜㅜ 3만원 이상 주 문 시 에 사용 할 수 있는 4천원 쿠폰 은 받아 볼 수 있어서 다운 받아서 스킨 케어 구입 했어요 뷰티 컬 리페 스타 선착순 할인 쿠폰 은 꼭 받으시길 그리고 결제 할 때 더블 적립 도 받아야죠 5만원 이상 결제 하신다면 네이버 페이 로 결제 하시 기를 추천 하는데요 네이버 페이 로 결제 하면 기본 2천 포인트 는 적립 되고요 삼 성 카드 나 우리카드 로 결제 하게

In [2]:
# Kiwi와 KeyBERT 모델 초기화
kiwi = Kiwi()
kw_model = KeyBERT()

# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and (pos.startswith('N') or pos.startswith('SL')):
            results.append(token)
    return results

# 각 행의 텍스트에 대해 명사 추출 및 키워드 추출
extracted_keywords = []
for text in tqdm(content_list):
    # 명사 추출
    nouns = noun_extractor(text)
    after_kiwi_text = ' '.join(nouns)
    
    # KeyBERT를 사용한 키워드 추출
    preprocessed_content_keywords = kw_model.extract_keywords(after_kiwi_text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=3)
    
    # 추출된 키워드 저장
    extracted_keywords.append(preprocessed_content_keywords)

# 결과 출력
print(extracted_keywords[:5])


100%|██████████| 3992/3992 [05:04<00:00, 13.10it/s]

[[('성공', 0.2497), ('설정', 0.2425), ('증정', 0.236)], [('런칭', 0.2316), ('영역', 0.2201), ('알갱이', 0.2199)], [('걱정', 0.2466), ('증정', 0.2402), ('선택', 0.2123)], [('라이프스타일', 0.2852), ('인생', 0.2821), ('과정', 0.2751)], [('걱정', 0.2453), ('증정', 0.2437), ('선인장', 0.2354)]]


In [3]:
# Extracting only the words from the extracted keywords
words_only = [[keyword[0] for keyword in keywords_list] for keywords_list in extracted_keywords]

# Creating a new DataFrame with the words only
keywords_df = pd.DataFrame({
    'Extracted_Keywords': words_only
})

keywords_df.head()

,Extracted_Keywords
0,"[성공, 설정, 증정]"
1,"[런칭, 영역, 알갱이]"
2,"[걱정, 증정, 선택]"
3,"[라이프스타일, 인생, 과정]"
4,"[걱정, 증정, 선인장]"


In [4]:
importance_df = pd.DataFrame()
importance_df['importance'] = data['likes'].values + data['comments'].values
importance_df.head()

,importance
0,25.0
1,0.0
2,26.0
3,11.0
4,12.0


In [5]:
importance_keyword_df = pd.concat([importance_df,keywords_df],axis=1)
importance_keyword_df.head()

,importance,Extracted_Keywords
0,25.0,"[성공, 설정, 증정]"
1,0.0,"[런칭, 영역, 알갱이]"
2,26.0,"[걱정, 증정, 선택]"
3,11.0,"[라이프스타일, 인생, 과정]"
4,12.0,"[걱정, 증정, 선인장]"


In [7]:
datetime_keyword_hashtags_importance_df = pd.concat([data[['datetime','hashtags']],importance_keyword_df], axis=1)
datetime_keyword_hashtags_importance_df.head()

,datetime,hashtags,importance,Extracted_Keywords
0,2023. 2. 11. 18:35,"['뷰티컬리페스타', '뷰티컬리', '뷰티컬리세일', '뷰티컬리추천', '뷰티컬리추...",25.0,"[성공, 설정, 증정]"
1,2022. 11. 1. 23:52,"['마켓컬리', '마켓컬리추천', '마켓컬리화장품', '뷰티컬리', '뷰티컬리러쉬'...",0.0,"[런칭, 영역, 알갱이]"
2,2022. 11. 13. 23:37,"['헤라블랙쿠션', '뷰티컬리', '마켓컬리', '샛별배송', '헤라쿠션']",26.0,"[걱정, 증정, 선택]"
3,2023. 8. 8. 20:10,"['뷰티컬리입점템', '마켓컬리추천상품', '라로제', '라로제토닉로션', '라로제...",11.0,"[라이프스타일, 인생, 과정]"
4,2023. 2. 15. 21:24,"['뷰티컬리', '뷰티컬리페스타', '뷰티컬리세일', '뷰티컬리추천', '뷰티컬리추...",12.0,"[걱정, 증정, 선인장]"


In [8]:
def convert_to_list(string):
    return string.strip("[]").replace("'", "").split(", ")

# apply 함수를 사용하여 데이터 변환
datetime_keyword_hashtags_importance_df['hashtags'] = datetime_keyword_hashtags_importance_df['hashtags'].apply(convert_to_list)


datetime_keyword_hashtags_importance_df

,datetime,hashtags,importance,Extracted_Keywords
0,2023. 2. 11. 18:35,"[뷰티컬리페스타, 뷰티컬리, 뷰티컬리세일, 뷰티컬리추천, 뷰티컬리추천템, 뷰티]",25.0,"[성공, 설정, 증정]"
1,2022. 11. 1. 23:52,"[마켓컬리, 마켓컬리추천, 마켓컬리화장품, 뷰티컬리, 뷰티컬리러쉬, 러쉬배쓰밤, 러...",0.0,"[런칭, 영역, 알갱이]"
2,2022. 11. 13. 23:37,"[헤라블랙쿠션, 뷰티컬리, 마켓컬리, 샛별배송, 헤라쿠션]",26.0,"[걱정, 증정, 선택]"
3,2023. 8. 8. 20:10,"[뷰티컬리입점템, 마켓컬리추천상품, 라로제, 라로제토닉로션, 라로제토너, 뷰티컬리]",11.0,"[라이프스타일, 인생, 과정]"
4,2023. 2. 15. 21:24,"[뷰티컬리, 뷰티컬리페스타, 뷰티컬리세일, 뷰티컬리추천, 뷰티컬리추천템]",12.0,"[걱정, 증정, 선인장]"
...,...,...,...,...
3987,2023. 4. 12. 10:36,"[안다로미네앱테크, 캐시워크, 돈버는퀴즈, 캐시워크정답, 안다로미네캐시워크, 앱테크...",6.0,"[추천, 선착순, 성공]"
3988,2023. 5. 16. 9:09,"[캐시워크, 캐시워크, 캐시워크정답, 캐시워크돈버는퀴즈, 캐시워크돈버는퀴즈정답, 앱...",7.0,"[걱정, 증정, 선착순]"
3989,2022. 11. 14. 15:13,"[뷰티컬리, 마켓컬리, 추천, 아로마티카, 비오템, 이니스프리, 달바]",11.0,"[마스터, 걱정, 망상]"
3990,2023. 4. 14. 12:05,"[캐시워크, 캐시워크돈버는퀴즈, 캐시워크정답, 캐시워크정답모음, 캐시워크쿠폰, 캐시...",6.0,"[성공, 선착순, 견적]"


In [16]:
datetime_keyword_hashtags_importance_df.to_csv('datetime_keyword_hashtags_importance_beauty.csv', encoding='utf-8', index=False)